In [1]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models


# This code was tested with TensorFlow v1.8
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 2.3.0


## Get the data
The data has been exported from BigQuery and is stored on GCS, and can be access directly via 

  `!gsutil cp gs://dataset-uploader/bbc/bbc-text.csv .`

It can also be accessed via https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv 

In [3]:
import re
TRAIN_FILEPATH = "../Translated/cleaned/train.csv"
TEST_FILEPATH = "../Translated/cleaned/test.csv"
SS_FILEPATH = "../data/SampleSubmission.csv"
VECTORS_FILEPATH = ""
data = pd.read_csv(TRAIN_FILEPATH)
test2 = pd.read_csv(TEST_FILEPATH)
ss = pd.read_csv(SS_FILEPATH)
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Every dataset is lower cased except for TREC
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r",", " ", string) 
    string = re.sub(r"!", " ", string) 
    string = re.sub(r"\(", " ", string) 
    string = re.sub(r"\)", " ", string) 
    string = re.sub(r"\?", " ", string) 
    string = re.sub(r"\s{2,}", " ", string)   
    string = re.sub(" \d+", " ", string)
    return  string.strip().lower()
data["Text"] =data.Text.apply(lambda x: clean_str(x))
test2["Text"] =test2.Text.apply(lambda x: clean_str(x))

In [4]:
data.head()

,ID,Text,Label
0,ID_AASHwXxg,mwangonde khansala wachinyamata akamati achiny...,POLITICS
1,ID_AGoFySzn,mcp siidakhutire ndi kalembera chipani cha mal...,POLITICS
2,ID_AGrrkBGP,bungwe la manepo lapempha boma liganizire anth...,HEALTH
3,ID_AIJeigeG,ndale zogawanitsa miyambo zanyanya si zachilen...,POLITICS
4,ID_APMprMbV,nanga wapolisi ataphofomoka masiku ano sichikh...,LAW/ORDER


In [5]:
data['Label'].value_counts()

POLITICS                279
SOCIAL                  152
RELIGION                147
LAW/ORDER               136
SOCIAL ISSUES           134
HEALTH                  127
ECONOMY                  86
FARMING                  78
SPORTS                   49
EDUCATION                43
RELATIONSHIPS            39
WILDLIFE/ENVIRONMENT     36
OPINION/ESSAY            26
LOCALCHIEFS              25
CULTURE                  23
WITCHCRAFT               16
MUSIC                    15
TRANSPORT                11
FLOODING                  7
ARTS AND CRAFTS           7
Name: Label, dtype: int64

In [6]:
train_size = int(len(data) * .9)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(data) - train_size))

Train size: 1292
Test size: 144


In [7]:
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

## Data preparation
There's some work to be done in order for our data to be ready for training.
1. First we'll split the data into training and test sets.
1. Then we'll tokenize the words (text), and then convert them to a numbered index. 
1. Next we'll do the same for the labels (categories), by using the `LabelEncoder` utility.
1. Finally, we'll convert the labels to a one-hot representation.

0       mwangonde khansala wachinyamata akamati achiny...
1       mcp siidakhutire ndi kalembera chipani cha mal...
2       bungwe la manepo lapempha boma liganizire anth...
3       ndale zogawanitsa miyambo zanyanya si zachilen...
4       nanga wapolisi ataphofomoka masiku ano sichikh...
                              ...                        
2051    kanyongolo wapempha oyimira milandu kuti atsat...
2052    amandimenya zikomo gogo ndine mtsikana wa zaka...
2053    apolisi athotha gulu la myp asilikali  a gulu ...
2054    mwambo wa ukwati wa chitonga mtundu wina uliwo...
2055    mwapasa autsa mapiri pamene pali kusamvana pak...
Name: Text, Length: 2056, dtype: object

In [8]:
train_cat, test_cat = train_test_split(data['Label'], train_size)
train_text, test_text = train_test_split(data['Text'], train_size)

In [27]:
max_words = 65000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, 
                                              char_level=False)

In [28]:
tokenize.fit_on_texts(pd.concat([data['Text'],test2['Text'] ], ignore_index = True)) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [35]:
tesst = tokenize.texts_to_matrix(test2['Text'])

In [29]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)

In [30]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [31]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (1292, 65000)
x_test shape: (144, 65000)
y_train shape: (1292, 20)
y_test shape: (144, 20)


## Train the model

Build the model using Keras layers and hyperparameters of your choosing. Then call `model.fit()`

In [31]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
Naive = naive_bayes.MultinomialNB()
Naive.fit(x_train,y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(x_test)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, y_test)*100)

Naive Bayes Accuracy Score ->  31.25


In [32]:
SVM = svm.SVC(C=0.5, kernel='linear', degree=3, gamma='auto')
SVM.fit(x_train,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(x_test)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, y_test)*100)

SVM Accuracy Score ->  54.861111111111114


In [14]:
# This model trains very quickly and 2 epochs are already more than enough
# Training for more epochs will likely lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 32
epochs = 5
drop_ratio = 0.5

In [32]:
# Build the model
from keras import backend as K 

# Do some code, e.g. train and save model

K.clear_session()
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

model = models.Sequential()
model.add(layers.Dense(4048, input_shape=(max_words,)))
# model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
# model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(2048))
# model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
# model.add(layers.Dense(512))
# # model.add(layers.BatchNormalization())
# model.add(layers.Activation('relu'))
# model.add(layers.Dense(128))
# # model.add(layers.BatchNormalization())
# model.add(layers.Activation('relu'))

# model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [33]:
# model.fit trains the model
# The validation_split param tells Keras what % of our training data should be used in the validation set
# You can see the validation loss decreasing slowly when you run this
# Because val_loss is no longer decreasing we stop training to prevent overfitting
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/5
37/37 [==============================] - 54s 1s/step - loss: 1.8917 - accuracy: 0.4509 - val_loss: 1.5256 - val_accuracy: 0.5538
Epoch 2/5
37/37 [==============================] - 51s 1s/step - loss: 0.2171 - accuracy: 0.9570 - val_loss: 1.6193 - val_accuracy: 0.6231
Epoch 3/5
37/37 [==============================] - 51s 1s/step - loss: 0.0253 - accuracy: 0.9974 - val_loss: 1.7997 - val_accuracy: 0.6154
Epoch 4/5
37/37 [==============================] - 52s 1s/step - loss: 0.0078 - accuracy: 0.9974 - val_loss: 1.7454 - val_accuracy: 0.6000
Epoch 5/5
37/37 [==============================] - 52s 1s/step - loss: 0.0031 - accuracy: 0.9983 - val_loss: 1.7420 - val_accuracy: 0.6000


## Evaluate the model
Evaluation is easy. Just call `model.evaluate()`.

In [34]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

5/5 [==============================] - 1s 143ms/step - loss: 1.7580 - accuracy: 0.6111
Test loss: 1.7580171823501587
Test accuracy: 0.6111111044883728


In [37]:
pred = model.predict(tesst)

In [50]:
# pred.idxmax(axis = 1)
# encoder.classes_
test2['Label'] = encoder.inverse_transform(np.argmax(pred,axis = 1))

In [54]:
sub = test2[['ID', 'Label']]

In [55]:
sub.to_csv('bbdtccopy1.csv', index=False)

In [44]:
encoder.inverse_transform(pred.max(axis = 1))

ValueError: y contains previously unseen labels: [0.12125547 0.17021933 0.17812623 0.20617856 0.21131323 0.21309534
 0.21882989 0.219577   0.22398123 0.22735713 0.23309939 0.23509465
 0.24311888 0.24525133 0.2535646  0.25413883 0.25433058 0.25569847
 0.25783992 0.25839826 0.2599425  0.2612511  0.2662768  0.26662397
 0.2668605  0.26713452 0.2687354  0.27048677 0.27711403 0.27719718
 0.27995712 0.28075048 0.2855997  0.28704923 0.29062906 0.29099706
 0.29197454 0.2932048  0.29380363 0.29774266 0.30222556 0.30292076
 0.3074179  0.30762085 0.3102441  0.31173643 0.3121684  0.31485343
 0.31811965 0.32010695 0.32030827 0.3277155  0.3284541  0.3295493
 0.3322034  0.33412695 0.33610868 0.3368298  0.3395701  0.34082395
 0.3408484  0.34141093 0.34177706 0.34336495 0.34398943 0.3499107
 0.3524968  0.3554892  0.35627943 0.35825643 0.35872695 0.3612791
 0.3620189  0.36241445 0.3712053  0.37784177 0.3841387  0.3859275
 0.38705185 0.38834545 0.38971168 0.3915831  0.395211   0.40200844
 0.40416992 0.407

## Hyperparameter tuning

This is a good time to go back and tweak some parameters such as `epoch`, `batch size`, `dropout ratio`, network structure, activation function, and others, to see if you can improve the accuracy.

In this particular case, to make it more challenging, I recommend reducing the max words of the call to `keras.preprocessing.text.Tokenizer`. This will reduce the number of words for each input sample, thus making it more challenging to accurately predict the category. (Notice that not all hyperparameters are necessarily inside the model. This is one such example.)

The default was up to 1000 words per article. See what happens when you reduce that number to 200 words, or 50 words, or even fewer. As the evaluation accuracy drops, the effects of your hyperparameter tuning will be more pronounced, with successful adjustments making meaningful improvements to the model performance.

To make this process easier to manage, I've encapulated the model definition and training and evaluation calls into one function call. You can add additional parameters as needed.

In [ ]:
def run_experiment(batch_size, epochs, drop_ratio):
  print('batch size: {}, epochs: {}, drop_ratio: {}'.format(
      batch_size, epochs, drop_ratio))
  model = models.Sequential()
  model.add(layers.Dense(2512, input_shape=(max_words,)))
  model.add(layers.Activation('relu'))
#   model.add(layers.Dropout(drop_ratio))
#   model.add(layers.Dense(512))
#   model.add(layers.Activation('relu'))
#   model.add(layers.Dropout(drop_ratio))
  model.add(layers.Dense(num_classes))
  model.add(layers.Activation('softmax'))

  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_split=0.1)
  score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=0)
  print('\tTest loss:', score[0])
  print('\tTest accuracy:', score[1])

In [ ]:
batch_size = 32
epochs = 2
drop_ratio = 0.2
run_experiment(batch_size, epochs, drop_ratio)

###Hyperparameter Search

You can also automate this process using for-loops and more sophiscated methods of deciding which combinations of hyperparameter values to try out.

Exhaustive search is generally not the most elegant way, this is mostly just for illustrative purposes.

In [ ]:
# Note: The data processed for this output had the max text length set to 400.
# for batch_size in range(10,31,10):
#   for epochs in range(3,15,5):
#     for drop_ratio in np.linspace(0.1, 0.5, 3):
#       run_experiment(batch_size, epochs, drop_ratio)

## Make some predictions
Take some samples from the test dataset and inspect some individual predictions, to ensure that things are sensible.


In [ ]:
# Here's how to generate a prediction on individual examples
text_labels = encoder.classes_ 

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_text.iloc[i][:50], "...")
    print('Actual label:' + test_cat.iloc[i])
    print("Predicted label: " + predicted_label + "\n")  

## (optional) Extra extra! Visualize the confusion matrix
This can help identify which areas were a challenge to get right, if the model is performing poorly.

In [ ]:
y_softmax = model.predict(x_test)

y_test_1d = []
y_pred_1d = []

for i in range(len(y_test)):
    probs = y_test[i]
    index_arr = np.nonzero(probs)
    one_hot_index = index_arr[0].item(0)
    y_test_1d.append(one_hot_index)

for i in range(0, len(y_softmax)):
    probs = y_softmax[i]
    predicted_index = np.argmax(probs)
    y_pred_1d.append(predicted_index)

In [ ]:
# This utility function is from the sklearn docs: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """

    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=30)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=22)
    plt.yticks(tick_marks, classes, fontsize=22)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontsize=25)
    plt.xlabel('Predicted label', fontsize=25)

In [ ]:
cnf_matrix = confusion_matrix(y_test_1d, y_pred_1d)
plt.figure(figsize=(24,20))
plot_confusion_matrix(cnf_matrix, classes=text_labels, title="Confusion matrix")
plt.show()